In [ ]:

import os

# Setting up the enviorenment for pytorch memory allocation. To avoid un allocated memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


import kagglehub
from sklearn.preprocessing import StandardScaler
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from skimage.transform import resize
import pandas as pd
from matplotlib.image import imread
from skimage.io import imread_collection
from PIL import Image
import seaborn as sns
from sklearn import decomposition, preprocessing, svm
import sklearn.metrics as metrics #confusion_matrix, accuracy_score
from time import sleep 
from tqdm.notebook import tqdm
import os
sns.set()

In [ ]:

# Download latest version
path = kagglehub.dataset_download("lukechugh/best-alzheimer-mri-dataset-99-accuracy")

print("Path to dataset files:", path)

In [ ]:
import glob

# Dataset that should go with Alzheimer test label
very_mild_test = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/Very Mild Impairment/*')
mild_test = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/Mild Impairment/*')
moderate_test = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/Moderate Impairment/*')

# Dataset that should go with Alzheimer train label
very_mild_train = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/train/Very Mild Impairment/*')
mild_train = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/train/Mild Impairment/*')
moderate_train = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/train/Moderate Impairment/*')


# Dataset without Alzheimer for test
non_test = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/*')

# Dataset without Alzheimer for train
non_train = glob.glob(r'/kaggle/input/d/lukechugh/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/train/No Impairment/*')


# **Viewing the Images**

In [ ]:
print(non_train[1])
def view_image(directory):
    img = mpimg.imread(directory)
    plt.imshow(img)
    plt.title(directory)
    plt.axis('off')
    print(f'Image shape:{img.shape}')
    return img

print('One of the data in Non Alzheimer Folder')

view_image(moderate_train[1])

# **Combining the dataset**

In [ ]:
#Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from einops import rearrange, repeat
from einops.layers.torch import Rearrange


# Data preprocessing transforms
def get_transforms(image_size=224):
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ])

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class AlzheimerDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        """
        Args:
            image_paths (list of str): List of file paths to the images.
            labels (list of int): List of labels corresponding to each image.
            transform (callable, optional): Transformations to be applied to the images.
        """
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # can turn it to rgb using .convert("RGB")

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        # Get the corresponding label
        label = self.labels[idx]

        return image, label


In [ ]:
from sklearn.model_selection import train_test_split

# Paths and labels
train_paths = very_mild_train + mild_train + moderate_train + non_train
train_labels = [1] * len(very_mild_train) + [2] * len(mild_train) + [3] * len(moderate_train) + [0] * len(non_train)

test_paths = very_mild_test + mild_test + moderate_test + non_test
test_labels = [1] * len(very_mild_test) + [2] * len(mild_test) + [3] * len(moderate_test) + [0] * len(non_test)

# Apply transforms
transform = get_transforms(image_size=224)

# Create datasets
train_dataset = AlzheimerDataset(train_paths, train_labels, transform=transform)
test_dataset = AlzheimerDataset(test_paths, test_labels, transform=transform)


# The RESNET model to train the data on

# 2. Initializing the RESNET model for the training

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

# CBAM Attention Module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x))

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = x * self.ca(x)
        x = x * self.sa(x)
        return x

# Updated ResNet101 with CBAM
class ResNet101_CBAM_MultiClass(nn.Module):
    def __init__(self, num_classes=4, pretrained=True, freeze_backbone=True):
        super().__init__()
        self.backbone = models.resnet50(pretrained=pretrained)

        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        # Inject CBAM after layer4
        self.cbam = CBAM(in_planes=2048)

        # Replace the FC layer
        self.backbone.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.cbam(x)  # Apply attention

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.backbone.fc(x)
        return x


# 1. Setting up the data

In [ ]:
from torch.utils.data import DataLoader

# Hyperparameters
BATCH_SIZE = 32
NUM_CLASSES = 4  # Non-demented, Very Mild, Mild, Moderate
IMAGE_SIZE = 224

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initializing the model
num_classes = 4 
model = ResNet101_CBAM_MultiClass(num_classes=4, pretrained=True, freeze_backbone=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

#using multiple gpu's in case they are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)

# 3. Defining the loss function and the optimizer

In [ ]:
import torch.optim as optim
from transformers import get_cosine_schedule_with_warmup
EPOCHS = 10
# Schedular for the warmup
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_loader)*EPOCHS)

In [ ]:
# Training function for one epoch
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        _, predicted = outputs.max(1)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Validation function
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            _, predicted = outputs.max(1)

            running_loss += loss.item()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = running_loss / len(dataloader)
    val_acc = 100 * correct / total
    return val_loss, val_acc


# 4. Loop for the training with the model

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=10):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        
        # Train the model for one epoch
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        print(f"Training - Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        
        # Validate the model
        val_loss, val_acc = validate(model, test_loader, criterion, device)
        print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
        
        # Step the scheduler based on validation loss
        scheduler.step(val_loss)
        
        # Monitor the learning rate
        for param_group in optimizer.param_groups:
            print(f"Learning Rate: {param_group['lr']}")


# 5. Loop for the evaluation while training

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np

def evaluate_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    test_loss = 0.0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # Compute loss
            if isinstance(criterion, nn.BCELoss):
                target = target.float()
                loss = criterion(output.squeeze(), target)
                predicted = (output.squeeze() > 0.5).float()
            else:
                loss = criterion(output, target)
                _, predicted = torch.max(output.data, 1)
            
            test_loss += loss.item()
            
            # Collect predictions and true labels
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    # Calculate average test loss
    test_loss /= len(test_loader)

    # Convert to NumPy arrays
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    # Calculate accuracy
    accuracy = 100 * (all_preds == all_targets).sum() / len(all_targets)

    # Generate classification report
    class_report = classification_report(all_targets, all_preds, target_names=["Non-Demented", "Very Mild", "Mild", "Moderate"])

    # Generate confusion matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Compute F1 score (macro-average)
    f1 = f1_score(all_targets, all_preds, average='macro')

    # Print results
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    print("\nClassification Report:\n", class_report)
    print("\nConfusion Matrix:\n", conf_matrix)
    print(f"\nMacro F1 Score: {f1:.2f}")

    return accuracy, test_loss, f1, class_report, conf_matrix


# Executing the training and evaluation functions

In [ ]:
# Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, epochs=10)

# Evaluate the model
evaluate_model(model, test_loader, criterion, device)

# Save the trained model to a file in the Kaggle working directory
model_save_path = "/kaggle/working/alzheimers_model.pth"
torch.save(model.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")


# Function to predict an image

In [ ]:
from PIL import Image
import torch
from torchvision import transforms


# Function to preprocess and predict
def predict_image(model, image_path, device, class_names):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    
    # Define transformations (same as used during training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ])
    
    # Preprocess the image
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Move to device
    image_tensor = image_tensor.to(device)
    
    # Set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        # Get predictions
        output = model(image_tensor)
        probabilities = torch.softmax(output[0], dim=0)  # Applying softmax
        predicted_class = probabilities.argmax().item()  # Geting the index of the predicted class
    
    # Print result
    print(f"Predicted category: {predicted_class}")
    print(f"Probabilities: {probabilities.cpu().numpy()}")
    
    return class_names[predicted_class]

# Define class names (replace with your actual classes)
class_names = ['Non Demented', 'Very Mild Demented', 'Mild Demented', 'Moderately Demented']

# Path to the image
image_path = non_test[2]

# Predict category
predicted_category = predict_image(model, image_path, device, class_names)
print(f"The Alzheimer's disease category is: {predicted_category}")
